In [1]:
import pandas as pd

df = pd.read_csv('_polygonDataExceptVertices.csv')
df = df.drop(['polygon_id', 'resolution'], axis=1)
df = df.drop(df[df.city != 'Oxnard'].index) 
df = df.drop(['city', 'projection_zone'], axis=1)
df.head()

,centroid_latitude,centroid_longitude,centroid_latitude_pixels,centroid_longitude_pixels,area_pixels,area_meters,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude,datum,jaccard_index
15416,34.243108,-119.160555,2118.488363,3428.374855,90.020804,1000.231156,621051913,34.248828,-119.171986,34.238028,-119.151981,NAD83,0.000000
15417,34.241204,-119.165305,2823.293951,2004.022414,11.117869,123.531876,621051913,34.248828,-119.171986,34.238028,-119.151981,NAD83,0.796921
15418,34.239659,-119.169852,3395.750380,640.194779,9.127378,101.415315,621051913,34.248828,-119.171986,34.238028,-119.151981,NAD83,0.791697
15419,34.239620,-119.169748,3410.340968,671.224025,20.557874,228.420819,621051913,34.248828,-119.171986,34.238028,-119.151981,NAD83,0.836574
15420,34.240401,-119.164998,3121.117334,2095.824496,10.289467,114.327411,621051913,34.248828,-119.171986,34.238028,-119.151981,NAD83,0.856176


In [2]:
df.value_counts('image_name').head(6)

image_name
621651901    323
621051897    185
619251901    160
620451901    103
620451909     96
621051877     84
dtype: int64

In [3]:
# Hecho manual, se puede automatizar

df = df.drop(df[df.image_name != '621051897'].index)
df = df.drop(['centroid_latitude', 'centroid_longitude', 'area_pixels', 'area_meters', 'image_name', 
             'nw_corner_of_image_latitude', 'nw_corner_of_image_longitude', 'se_corner_of_image_latitude',
             'se_corner_of_image_longitude', 'datum', 'jaccard_index'], axis=1)
df

,centroid_latitude_pixels,centroid_longitude_pixels
15804,65.617123,1172.430705
15805,142.778505,1193.402185
15806,102.563952,1399.142734
15807,634.021988,304.092654
15808,637.627039,438.649723
...,...,...
15984,2589.805821,3228.477234
15985,2608.745161,3110.761974
15986,2606.141566,3233.910727
15987,1166.336156,4192.005515


In [4]:
import numpy as np

df_array = np.array(df)

labels = np.zeros(2500, int) 

df_array_latitude = np.ceil( df_array[:,0] / 80 )
df_array_longitude = np.ceil( df_array[:,1] / 120 )

df_array_1 = np.stack( (df_array_latitude, df_array_longitude) , axis=1).astype(int)

df_array_1_unique = np.unique(df_array_1, axis=0)

import itertools

comb = np.arange(1, 51)
mesh = np.array(np.meshgrid(comb, comb)).T.reshape(-1, 2)

indexes = []
for _ in df_array_1_unique:
    indexes.append(np.argwhere(np.all((mesh-np.array(_))==0, axis=1)))
    
for _ in range(0, len(indexes)):
    labels[indexes[_]] = 1

In [10]:
print(mesh)
print('Dimensiones de mesh: ', mesh.shape)

[[ 1  1]
 [ 1  2]
 [ 1  3]
 ...
 [50 48]
 [50 49]
 [50 50]]
Dimensiones de mesh:  (2500, 2)


In [14]:
df_array_1_unique[0:5]

array([[ 1, 10],
       [ 2, 10],
       [ 2, 12],
       [ 8,  3],
       [ 8,  4]])

In [13]:
np.unique(labels, return_counts=True)

(array([0, 1]), array([2459,   41]))

In [11]:
labels_tot = labels # Sólo la primera vez (621651901)
#labels_tot = np.concatenate((labels_tot, labels)) # Quitar la primera vez
labels_tot

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
labels_tot.shape

(2500,)

In [29]:
import pandas as pd

pd.DataFrame(labels).to_csv('_labels_train.csv')

### Importar el vector de los _labels_ y el de las imágenes

In [4]:
numbers = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
          '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
          '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
          '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
          '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']

In [30]:
import os

os.chdir('621651901')

In [31]:
import PIL
from PIL import Image
import numpy as np

pixels_data = []
for i in numbers:
    for j in numbers:
        img = Image.open('621651901' + '_' + i + '_' + j + '.png')
        pixels = np.asarray(img)
        pixels_data = np.append(pixels_data, pixels)
pixels_data = pixels_data.astype(int).reshape(2500, 80, 120)

In [32]:
import pandas as pd

pd.DataFrame(pixels_data.reshape(24000000,1)).to_csv('_data_train.csv')

### Exportar los vectores de las imágenes y los _labels_ a un excel

In [221]:
labels_data = labels_tot[0:2500]

In [225]:
print(pixels_data.shape)
print(labels_data.shape)

(2500, 80, 120)
(2500,)


In [226]:
import pandas as pd

pd.DataFrame(labels_data).to_csv('_labels_data.csv')
pd.DataFrame(pixels_data.reshape(24000000,1)).to_csv('_pixels_data.csv')